In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.diagnostic as smd
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression, Lasso
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score
import sklearn.metrics as metrics
from sklearn.linear_model import Ridge
import seaborn as sns
import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot
from yellowbrick.regressor import ResidualsPlot
import scipy.stats as stats
import statsmodels.stats.diagnostic as diag
from scipy.stats import normaltest
import random
from sklearn.feature_selection import SequentialFeatureSelector #Para stepwise
from sklearn.linear_model import RidgeCV, Ridge #Para Ridge
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate

plt.rcParams['figure.figsize'] = (15, 9)
plt.style.use('ggplot')
random.seed(123)

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
houses_df = pd.read_csv('train.csv', encoding='latin1')

houses_df.head()

In [ ]:
#exploratory data analysis
houses_df.describe()



plt.scatter(houses_df['LotArea'], houses_df['SalePrice'])
plt.xlabel('Lot Area (ft^2)')
plt.ylabel('Sale Price ($)')
plt.show()



In [ ]:
numeric_columns = ['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
 '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF',
 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']

numeric = houses_df[numeric_columns]
numeric

In [ ]:
numeric = numeric.drop(['BsmtFinSF2', '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath', 
            'HalfBath', 'Fireplaces', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch','ScreenPorch', 
            'PoolArea', 'MiscVal'], axis=1)


In [ ]:
numeric = numeric[numeric["SalePrice"] < 700000]
numeric = numeric[numeric["LotArea"] < 100000]

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir el conjunto de datos en características (X) y la variable objetivo (y)
X = numeric.drop("SalePrice", axis=1)
y = numeric["SalePrice"]

# Dividir los datos en conjunto de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Si se proporciona un conjunto de datos de prueba adicional, se puede usar como conjunto de validación

# Si no, se puede dividir aún más el conjunto de prueba para tener un conjunto de validación separado
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # División 80/20 entre entrenamiento y validación

# Imprimir los tamaños de los conjuntos resultantes
print("Tamaño del conjunto de entrenamiento:", len(X_train))
print("Tamaño del conjunto de validación:", len(X_val))
print("Tamaño del conjunto de prueba:", len(X_test))


In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Variable predictora
X = X_train['GrLivArea']

# Añadir una constante para el término de intercepción
X = sm.add_constant(X)

# Modelo de regresión lineal
model = sm.OLS(y_train, X).fit()

# Resumen del modelo
print(model.summary())

# Hacer predicciones
predictions = model.predict(X)

# Gráfico de dispersión y línea de regresión
plt.scatter(X_train['GrLivArea'], y_train, color='blue', label='Actual')
plt.plot(X_train['GrLivArea'], predictions, color='red', linewidth=2, label='Predicted')
plt.xlabel('GrLivArea')
plt.ylabel('Sale Price')
plt.title('Univariate Linear Regression')
plt.legend()
plt.show()

# Gráfico de residuos
residuals = y_train - predictions
plt.scatter(X_train['GrLivArea'], residuals)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('GrLivArea')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()


In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Añadir una constante para el término de intercepción
X_train_const = sm.add_constant(X_train)

# Modelo de regresión lineal
model = sm.OLS(y_train, X_train_const).fit()

# Resumen del modelo
print(model.summary())

# Hacer predicciones
predictions = model.predict(X_train_const)

# Gráfico de dispersión entre valores reales y predichos
plt.scatter(y_train, predictions)
plt.xlabel('Actual Sale Price')
plt.ylabel('Predicted Sale Price')
plt.title('Actual vs Predicted Sale Price')
plt.show()

# Gráfico de residuos
residuals = y_train - predictions
plt.scatter(predictions, residuals)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Predicted Sale Price')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE


# Dividir el conjunto de datos en características (X) y la variable objetivo (y)
X = numeric.drop("SalePrice", axis=1)
y = numeric["SalePrice"]

# Dividir los datos en conjunto de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Si se proporciona un conjunto de datos de prueba adicional, se puede usar como conjunto de validación

# Si no, se puede dividir aún más el conjunto de prueba para tener un conjunto de validación separado
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # División 80/20 entre entrenamiento y validación

# Imprimir los tamaños de los conjuntos resultantes
print("Tamaño del conjunto de entrenamiento:", len(X_train))
print("Tamaño del conjunto de validación:", len(X_val))
print("Tamaño del conjunto de prueba:", len(X_test))

# Añadir una constante para el término de intercepción
X_train_const = sm.add_constant(X_train)

# Modelo de regresión lineal
model = sm.OLS(y_train, X_train_const).fit()



In [ ]:
# 9.
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for each feature
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif["features"] = X_train.columns

print(vif)

# Calculate R-squared
r_squared = model.rsquared
print('R-squared:', r_squared)

# Calculate Adjusted R-squared
adjusted_r_squared = model.rsquared_adj
print('Adjusted R-squared:', adjusted_r_squared)
# Check for overfitting
predictions_val = model.predict(sm.add_constant(X_val))
mse_train = mean_squared_error(y_train, predictions)
mse_val = mean_squared_error(y_val, predictions_val)

print('Training MSE:', mse_train)
print('Validation MSE:', mse_val)

In [ ]:
# 9 y 10


# elegimos variables con base a lasso
# Create a lasso regressor
model = Lasso(alpha=0.1)

# Fit the regressor to the data
rfe = RFE(model, n_features_to_select=6)

# Fit the model
fit = rfe.fit(X_train, y_train)

print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)

# Seleccionar las características seleccionadas

selected_features = X_train.columns[fit.support_]
print("Selected features: ", selected_features)

X = numeric[selected_features]
y = numeric["SalePrice"]


# Eliminamos las variables con VIF alto
# X=numeric
# X = numeric.drop(["SalePrice", ], axis=1)  
# y = numeric["SalePrice"]

# Juntar OverQual y OverCond (promedio)

# X["OverallQualCond"] = (X["OverallQual"] + X["OverallCond"]) / 2
# X = X.drop(["OverallQual", "OverallCond"], axis=1)

# Dividir los datos en conjunto de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Si no, se puede dividir aún más el conjunto de prueba para tener un conjunto de validación separado
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # División 80/20 entre entrenamiento y validación

# Imprimir los tamaños de los conjuntos resultantes
print("Tamaño del conjunto de entrenamiento:",  X_train.shape)
print("Tamaño del conjunto de validación:",  X_val.shape)
print("Tamaño del conjunto de prueba:" ,X_test.shape)

# Añadir una constante para el término de intercepción
X_train_const = sm.add_constant(X_train)

# Modelo de regresión lineal
model = sm.OLS(y_train, X_train_const).fit()

residuals = y_train - model.predict(X_train_const)

# Crear la gráfica de residuos
plt.scatter(model.predict(X_train_const), residuals)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Predicted Sale Price')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()

y_pred = model.predict(sm.add_constant(X_train))

# Calcular el MSE
mse = mean_squared_error(y_train, y_pred)
print("MSE:", mse)


# Calcular el RMSE
rmse = np.sqrt(mse)
print("RMSE:", rmse)

In [ ]:
# Añadir una constante al conjunto de prueba
from sklearn.metrics import mean_absolute_error

print(X_test.shape)

X_test_const = sm.add_constant(X_test)

# Calcular las predicciones del modelo en el conjunto de prueba
y_pred_test = model.predict(X_test_const)

# Calcular el MSE en el conjunto de prueba
mse_test = mean_squared_error(y_test, y_pred_test)
print("MSE (test):", mse_test)

# Calcular el MAE en el conjunto de prueba
mae_test = mean_absolute_error(y_test, y_pred_test)
print("MAE (test):", mae_test)

# Calcular el RMSE en el conjunto de prueba
rmse_test = np.sqrt(mse_test)
print("RMSE (test):", rmse_test)

import matplotlib.pyplot as plt

# Crear un gráfico de dispersión de los valores reales vs las predicciones
plt.scatter(y_test, y_pred_test)
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.title('Valores Reales vs Predicciones')

# Dibujar una línea diagonal (indicando un modelo perfecto)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')

plt.show()

In [ ]:

# Try a different model
from sklearn.model_selection import cross_val_score


model = RandomForestRegressor(n_estimators=100, random_state=42)

# Use cross-validation
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Take the square root and make positive (because cross_val_score returns negative values for MSE)
rmse_scores = np.sqrt(-scores)

print("RMSE Scores:", rmse_scores)
print("RMSE Mean:", rmse_scores.mean())
print("RMSE Standard Deviation:", rmse_scores.std())

In [ ]:
from sklearn.feature_selection import RFE

model = Lasso(alpha=0.1)
rfe = RFE(model, n_features_to_select=3)
fit = rfe.fit(X_train, y_train)

print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)

In [ ]:

# Define the model
model = Lasso(alpha=0.1)

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred_test = model.predict(X_test)

# Calculate MSE and RMSE
mse_test = mean_squared_error(y_test, y_pred_test)
print("MSE (test):", mse_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
print("MAE (test):", mae_test)
rmse_test = np.sqrt(mse_test)
print("RMSE (test):", rmse_test)

# Plot the predictions
plt.scatter(y_test, y_pred_test)
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.title('Valores Reales vs Predicciones')

# Dibujar una línea diagonal (indicando un modelo perfecto)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')